In [3]:
import pandas as pd
import numpy as np
df =pd.read_csv("test_data1_clean.csv")
df.head()
df.shape

(44, 6)

In [7]:
df =pd.read_csv("test_data2_clean.csv")
df.head()
df.shape

(44, 9)

In [8]:
df =pd.read_csv("combined_clean.csv")
df.head()
df.shape

(88, 13)